# Test approximations
The idea is to have a basic workflow to test all the approximations ($\Delta_x$ and $F_x$) vs the ground truth for each system in ODEBench.

In [11]:
# lots of necessary imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re as regex
import seaborn as sns
import sympy

from scipy import integrate
from sklearn.metrics import r2_score
from sympy import parse_expr

# set stule for plots
sns.set_style('darkgrid')

In [12]:
# now, let's pick an ODE system from ODEBench
odebench_file_name = "../data/odebench/all_odebench_trajectories.json"
system_id = 14 # we can change this

# read the ODEBench file with all the trajectories
odebench = None
with open(odebench_file_name, "r") as fp :
    odebench = json.load(fp)
    
# pick the system with the appropriate system id
system = [system for system in odebench if system["id"] == system_id][0]

# get the names of the state variables
state_variables = regex.findall("([0-9|a-z|\_]+)\:\s+", system["var_description"])
#print(state_variables)

# associate each variable with the expression of its derivative
equations = {state_variables[i] : sympy.sympify(system["substituted"][0][i])
                     for i in range(0, len(state_variables))}
print(equations)

# get the trajectories
trajectories = system["solutions"][0] # for some reason, another list with 1 element (...) 
print("Found a total of %d trajectories!" % len(trajectories))

# we only consider one trajectory, for the moment
print("Converting the first trajectory to a DataFrame...")
trajectory = trajectories[0]
dictionary_trajectory = {}
dictionary_trajectory["t"] = trajectory["t"]

for v in range(0, len(state_variables)) :
    dictionary_trajectory[state_variables[v]] = trajectory["y"][v]
df_trajectory = pd.DataFrame.from_dict(dictionary_trajectory)
print(df_trajectory)

{'x_0': -0.9*x_0**2/(x_0**2 + 449.44) + 0.78*x_0*(1 - 0.0123456790123457*x_0)}
Found a total of 2 trajectories!
Converting the first trajectory to a DataFrame...
             t        x_0
0     0.000000   2.760000
1     0.067114   2.901931
2     0.134228   3.050817
3     0.201342   3.206967
4     0.268456   3.370693
..         ...        ...
145   9.731544  78.561747
146   9.798658  78.627607
147   9.865772  78.690294
148   9.932886  78.749955
149  10.000000  78.806732

[150 rows x 2 columns]


In [13]:
# now, we compute the approximation(s); first, $\Delta_t$, using built-in functions from another package
from pysindy.differentiation import FiniteDifference, SmoothedFiniteDifference

fd = FiniteDifference(order=2)
dy_dt = fd._differentiate(df_trajectory[state_variables].values, 
                                      df_trajectory["t"].values)

In [14]:
# prepare another pandas DataFrame
dy_dt_column_names = [v + "_dxdt" for v in state_variables]
# most of it will be a copy of the previous one
df_dy_dt = df_trajectory.copy()
for i in range(0, len(state_variables)) :
    df_dy_dt[dy_dt_column_names[i]] = dy_dt[:,i]
print(df_dy_dt)

             t        x_0  x_0_dxdt
0     0.000000   2.760000  2.062964
1     0.067114   2.901931  2.166589
2     0.134228   3.050817  2.272519
3     0.201342   3.206967  2.383076
4     0.268456   3.370693  2.498379
..         ...        ...       ...
145   9.731544  78.561747  1.006105
146   9.798658  78.627607  0.957676
147   9.865772  78.690294  0.911491
148   9.932886  78.749955  0.867462
149  10.000000  78.806732  0.824488

[150 rows x 3 columns]


In [15]:
from sympy.utilities.lambdify import lambdify
from sklearn.metrics import r2_score

for state_variable, equation in equations.items() :
    # now, evaluate the ground truth for $\Delta_x$; this should be exactly the same as the original x'(t)
    equation_symbols = [c for c in df_dy_dt.columns if not c.endswith("_dxdt")]
    #print(equation_symbols)
    symbol_values = [df_dy_dt[c].values for c in equation_symbols]
    #print(symbol_values)
    
    # lambdify equation and get values
    equation_values = lambdify(equation_symbols, equation)(*symbol_values)
    print(equation_values)
    ground_truth = df_dy_dt[state_variable + "_dxdt"].values

    # compute R2 score
    r2_value = r2_score(ground_truth, equation_values)
    print("For state variable \"%s\", R2=%.4f" % (state_variable, r2_value))

[ 2.06444539  2.16586009  2.27174984  2.38226317  2.49754253  2.61773213
  2.74297323  2.8734043   3.00916148  3.1503729   3.29716209  3.44964782
  3.60794181  3.77213885  3.9423292   4.11859515  4.30100863  4.48962892
  4.68448397  4.88558874  5.0929447   5.30653486  5.52631248  5.75221106
  5.98414455  6.22200339  6.46563647  6.71486724  6.96949586  7.22929352
  7.49398188  7.76325066  8.03676198  8.31414373  8.59496773  8.87876764
  9.16504603  9.45326737  9.74283959 10.03312678 10.32345639 10.61311305
 10.90133943 11.1873392  11.47027618 11.74927758 12.02343743 12.29182783
 12.55350442 12.80750047 13.05283829 13.28853648 13.51362402 13.72713995
 13.92814113 14.11571213 14.28897657 14.44711004 14.58933699 14.714943
 14.82328305 14.91379312 14.98599216 15.03948504 15.07396981 15.08924209
 15.08519743 15.06183083 15.0192379  14.95761402 14.87725202 14.77854073
 14.66195984 14.52807301 14.37752078 14.2110116  14.02932524 13.83330756
 13.62384685 13.4018708  13.16833651 12.92422651 12.6